# 단어 임베딩의 원리와 gensim.word2vec 사용법

단어 임베딩(Word Embedding)이란 텍스트를 구성하는 하나의 단어를 수치화하는 방법의 일종이다.

텍스트 분석에서 흔히 사용하는 방식은 단어 하나에 인덱스 정수를 할당하는 Bag of Words 방법이다. 이 방법을 사용하면 문서는 단어장에 있는 단어의 갯수와 같은 크기의 벡터가 되고 단어장의 각 단어가 그 문서에 나온 횟수만큼 벡터의 인덱스 위치의 숫자를 증가시킨다.

즉 단어장이 "I", "am", "a", "boy", "girl" 다섯개의 단어로 이루어진 경우 각 단어에 다음과 같이 숫자를 할당한다.

```
"I": 0
"am": 1
"a": 2
"boy": 3 
"girl": 4
```

이 때 "I am a girl" 이라는 문서는 다음과 같이 벡터로 만들 수 있다.

$$ [1 \; 1 \; 1 \; 0 \; 1] $$

단어 임베딩은 하나의 단어를 하나의 인덱스 정수가 아니라 실수 벡터로 나타낸다. 예를 들어 2차원 임베딩을 하는 경우 다음과 같은 숫자 벡터가 될 수 있다.

```
"I": (0.3, 0.2)
"am": (0.1, 0.8)
"a": (0.5, 0.6)
"boy": (0.2, 0.9) 
"girl": (0.4, 0.7)
```

단어 임베딩이 된 경우에는 각 단어 벡터를 합치거나(concatenation) 더하는(averaging, normalized Bag of Words) 방식으로 전체 문서의 벡터 표현을 구한다.

## Feed-Forward 신경망 언어 모형 (Neural Net Language Model)

이러한 단어 임베딩은 신경망을 이용하여 언어 모형을 만들려는 시도에서 나왔다. 자세한 내용은 다음 논문을 참고한다.

* "A Neural Probabilistic Language Model", Bengio, et al. 2003
  * http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf

* "Efficient Estimation of Word Representations in Vector Space", Mikolov, et al. 2013
  * https://arxiv.org/pdf/1301.3781v3.pdf
  
* "word2vec Parameter Learning Explained", Xin Rong, 
   * http://www-personal.umich.edu/~ronxin/pdf/w2vexp.pdf

V개의 단어를 가지는 단어장이 있을 때, 단어를 BOW 방식으로 크기 V인 벡터로 만든 다음 다음 그림과 같이 하나의 은닉층(Hidden Layer)을 가지는 신경망을 사용하여 특정 단어 열(word sequence)이 주어졌을 때 다음에 나올 단어를 예측하는 문제를 생각해 보자. 입력과 출력은 모두 BOW 방식으로 인코딩되어 있다.


<img src="https://datascienceschool.net/upfiles/cd100ec8d3d6476e9522ead4c2acf6a2.png" style="width: 50%;">

<small>이미지 출처: "word2vec Parameter Learning Explained", Xin Rong</small>

입력 $x$가 들어가면 입력 가중치 행렬 $W^T$이 곱해져서 은닉층 벡터 $h$가 되는데 $x$가 one-hot-encoding 된 값이므로 $h$ 벡터는 입력 가중치 행렬  $W$의 행 하나가 된다. 

$$ h = W^T x = v^T_i $$

여기에서 $i$는 입력 벡터 $x$ 의 값이 1인 원소의 인덱스이다. 즉, BOW 단어장에서 $i$번째 단어를 뜻한다.

벡터 $h$는 다시 출력 가중치 행렬 $W'^T$와 곱해져서 출력 벡터 $y$가 된다. 

$$ y = W'^T h $$

출력 가중치 행렬 $W'$의 $j$번째 열을 $v_j$라고 하면 출력 벡터 $y$의 $j$번째 원소의 값은 다음과 같다.

$$ y_j = v_j'^T h $$

가중치 행렬을 갱신하는 최적화 공식을 살펴본다. 자세한 유도과정은 논문을 참조한다.

우선 출력 가중치 행렬의 갱신 공식은 다음과 같다.

$$ v_j'^{\text{(new)}} = v_j'^{\text{(old)}} - \eta \cdot e_j \cdot h = v_j'^{\text{(old)}} - \eta \cdot e_j \cdot v_i^T $$ 

이 식에서 $\eta$는 최적화 스텝 사이즈, $e_j$는 출력 오차가 된다. 이 공식에 따르면 벡터 $v'_j$는 $v_j$ 방향으로 수렴해 간다. 즉, $i$번째 단어와 $j$번째 단어가 연속하는 관계라면 $v'_j$가 $v_i$와 유사한 위치로 수렴한다는 뜻이다.

다음으로 입력 가중치 행렬의 갱신 공식은 다음과 같다.

$$ v_i^{\text{(new)}} = v_i^{\text{(old)}} - \eta \sum_k e_j \cdot w'_{ik}$$ 

이 공식에 따르면 벡터 $v_i$는 여러 $v'_k$ 벡터의 가중합으로 수렴해 간다. 이렇게 단어간의 관계에 의해 $i$번째 단어를 뜻하는 $v_i$의 값들이 연관성을 가지게 되는데 이 $v_i$ 벡터 값을 해당 단어에 대한 **분산 표현 (distributed representation)** , **벡터 표현 (vector representation)** 또는 **단어 임베딩 (word embedding)**이라고 한다.

<img src="https://www.tensorflow.org/versions/r0.11/images/linear-relationships.png" style="width: 100%;">

<small>이미지 출처: https://www.tensorflow.org/versions/master/tutorials/word2vec/index.html</small>

## CBOW (Continuous Bag of Words) Embedding

위의 방식은 하나의 단어로부터 다음에 오는 단어를 예측하는 문제였다. 이러한 문제를 단어 하나짜리 문맥(single-word context)를 가진다고 한다. 

CBOW (Continuous Bag of Words) 방식은 복수 단어 문맥(multi-word context)에 대한 문제 즉, 여러개의 단어를 나열한 뒤 이와 관련된 단어를 추정하는 문제이다. 즉, 문자에서 나오는 $n$개의 단어 열로부터 다음 단어를 예측하는 문제가 된다. 예를 들어

> the quick brown fox jumped over the lazy dog

라는 문장에서 (`the`, `quick`, `brown`) 이라는 문맥이 주어지면 `fox`라는 단어를 예측해야 한다.

CBOW는 다음과 같은 신경망 구조를 가진다. 여기에서 각 문맥 단어를 은닉층으로 투사하는 가중치 행렬은 모든 단어에 대해 공통으로 사용한다.

<img src="https://datascienceschool.net/upfiles/3cdbdbfe1c8a4742aaf2e0c40917948f.png" style="width: 50%;">

<small>이미지 출처: "word2vec Parameter Learning Explained", Xin Rong</small>

## Skip-Gram Embedding

Skip-Gram 방식은 CBOW 방식과 반대로 특정한 단어로부터 문맥이 될 수 있는 단어를 예측한다. 보통 입력 단어 주변의 $k$개 단어를 문맥으로 보고 예측 모형을 만드는데 이 $k$ 값을 window size 라고 한다.

위 문장에서 window size $k=1$인 경우,

* `quick` -> `the`
* `quick` -> `brown`
* `brown` -> `quick`
* `brown` -> `fox`

과 같은 관계를 예측할 수 있어야 한다.

<img src="https://datascienceschool.net/upfiles/8f8ebfa0ebb34eb584d24d59fe60a12d.png" style="width: 50%;">

<small>이미지 출처: "word2vec Parameter Learning Explained", Xin Rong</small>

## word2vec

word2vec은 CBOW 방식과 Skip-Gram 방식의 단어 임베딩을 구현한 C++ 라이브러리로 구글에 있던 Mikolov 등이 개발하였다.

파이썬에서는 gensim이라는 패키지에 `Word2Vec`이라는 클래스로 구현되어 있다. nltk의 영화 감상 corpus를 기반으로 `Word2Vec` 사용법을 살펴보자.

우선 단어 임베딩을 위한 코퍼스를 만든다. 코퍼스는 리스트의 리스트 형태로 구현되어야 한다. 내부 리스트는 하나의 문장을 이루는 단어 열이 된다.

In [24]:
from nltk.corpus import movie_reviews
sentences = [list(s) for s in movie_reviews.sents()]

In [29]:
sentences[0]

[u'plot',
 u':',
 u'two',
 u'teen',
 u'couples',
 u'go',
 u'to',
 u'a',
 u'church',
 u'party',
 u',',
 u'drink',
 u'and',
 u'then',
 u'drive',
 u'.']

다음으로 이 코퍼스를 입력 인수로 하여 Word2Vec 클래스 객체를 생성한다. 이 시점에 트레이닝이 이루어진다.

In [33]:
from gensim.models.word2vec import Word2Vec

In [34]:
%%time
model = Word2Vec(sentences)

CPU times: user 12.2 s, sys: 400 ms, total: 12.6 s
Wall time: 7.33 s


트레이닝이 완료되면 `init_sims` 명령으로 필요없는 메모리를 unload 시킨다.

In [66]:
model.init_sims(replace=True)

이제 이 모형에서 다음과 같은 메서드를 사용할 수 있다. 보다 자세한 내용은 https://radimrehurek.com/gensim/models/word2vec.html 를 참조한다.

* `similarity` : 두 단어의 유사도 계산
* `most_similar` : 가장 유사한 단어를 출력

In [67]:
model.similarity('actor', 'actress')

0.87472425755991945

In [68]:
model.similarity('he', 'she')

0.85470770334392587

In [69]:
model.similarity('actor', 'she')

0.21756392610362227

In [70]:
model.most_similar("villain")

[(u'hero', 0.7978197932243347),
 (u'doctor', 0.7952470779418945),
 (u'actress', 0.7806568145751953),
 (u'performer', 0.775442361831665),
 (u'charming', 0.7602461576461792),
 (u'impression', 0.7583950757980347),
 (u'commoner', 0.7538788318634033),
 (u'janitor', 0.7536816000938416),
 (u'dude', 0.7528475522994995),
 (u'genius', 0.7506694793701172)]

`most_similar` 메서드는 `positive` 인수와 `negative` 인수를 사용하여 다음과 같은 단어간 관계도 찾을 수 있다.

> actor + he - actress = she

In [71]:
model.most_similar(positive=['actor', 'he'], negative='actress', topn=1)

[(u'she', 0.2471558153629303)]

이번에는 네이버 영화 감상 코퍼스를 사용하여 한국어 단어 임베딩을 해보자.

In [74]:
import codecs

def read_data(filename):
    with codecs.open(filename, encoding='utf-8', mode='r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]   # header 제외
    return data

train_data = read_data('/home/dockeruser/data/nsmc/ratings_train.txt')

In [75]:
from konlpy.tag import Twitter
tagger = Twitter()

def tokenize(doc):
    return ['/'.join(t) for t in tagger.pos(doc, norm=True, stem=True)]

train_docs = [row[1] for row in train_data]
sentences = [tokenize(d) for d in train_docs]

In [76]:
from gensim.models import word2vec
model = word2vec.Word2Vec(sentences)
model.init_sims(replace=True)

In [79]:
model.similarity(*tokenize(u'악당 영웅'))

0.6062297706048696

In [84]:
model.similarity(*tokenize(u'악당 감동'))

-0.0041346659756955097

In [81]:
from konlpy.utils import pprint
pprint(model.most_similar(positive=tokenize(u'배우 남자'), negative=tokenize(u'여배우'), topn=1))

[(여자/Noun, 0.6258430480957031)]


더 많은 한국어 코퍼스를 사용한 단어 임베딩 모형은 다음 웹사이트에서 테스트해 볼 수 있다.
* http://w.elnn.kr/